In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
path_time = '/kaggle/input/behav-company-img/dataframe/Behav_Company_img.csv'

In [ ]:
import pandas as pd
processed_df = pd.read_csv(path_time)
processed_df

In [ ]:
!pip install -U sentence-transformers

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

def load_mpnet_model(model_name='all-mpnet-base-v2', device='cuda'):
    model = SentenceTransformer(model_name).to(device)
    return model

def generate_text_embeddings(model, text_batch, device='cuda'):
    embeddings = model.encode(text_batch, convert_to_tensor=True).to(device)
    return embeddings

def generate_embeddings_for_text_dataframe(model, dataframe, text_column, batch_size=8, device='cuda'):
    embeddings_list = []

    for i in tqdm(range(0, len(dataframe), batch_size)):
        text_batch = dataframe[text_column].iloc[i:i + batch_size].tolist()

        embeddings_batch = generate_text_embeddings(model, text_batch, device)
        embeddings_list.append(embeddings_batch)

    embeddings_tensor = torch.cat(embeddings_list, dim=0)

    return embeddings_tensor

In [ ]:
def mpNet(df):
    text_column = 'Text'
    df[text_column] = df['username'] + df['inferred company'] + df['content_processed']

    batch_size = 8
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    mpnet_model = load_mpnet_model(device = device)
    embeddings_tensor = generate_embeddings_for_text_dataframe(mpnet_model, df, text_column, batch_size, device = device)

    print("Tensor of Text Embeddings:")
    print(embeddings_tensor)
    return embeddings_tensor

text_mpnet_emb = mpNet(processed_df)

In [ ]:
text_mpnet_emb.shape

In [ ]:
import numpy as np
text_mpnet_emb_np = text_mpnet_emb.cpu().numpy()
np.save('/kaggle/working/MPNET_Embeds_Company.npy', text_mpnet_emb_np)